In [138]:
try:
    import geopandas as gpd
except:
    !pip install geopandas
    import geopandas as gpd
import numpy as np
import networkx as nx
import pandas as pd
import os
from shapely.geometry import LineString


# Define the path explicitly for interactive environments
data_dir = "../ori_data/Coquitlam_Drainage_SHP"

# Load manholes (nodes) and storm mains (edges)
manholes_path = os.path.join(data_dir, "Storm_Manholes.shp")
mains_path = os.path.join(data_dir, "Storm_Mains.shp")

manholes = gpd.read_file(manholes_path)
mains = gpd.read_file(mains_path)




In [77]:
manholes.columns

Index(['GIS_ID', 'FCODE', 'STATUS', 'DESC', 'LOCATION', 'PARENT', 'DESC_DTL1',
       'DESC_DTL2', 'DESC_DTL3', 'INSTALL_DT', 'EXTRACT_DT', 'MAPSHEET',
       'ASSET_ID', 'INS_DIAM', 'ASBUILT', 'MH_TYPE', 'RIM_ELEV', 'INV_ELEV',
       'DFCDESGATE', 'DFCACTGATE', 'DFCTURNS', 'DFCDESFLOW', 'geometry'],
      dtype='object')

# Look at data

In [97]:
# check if INS_DIAM and SEC_DIAM are the same for each row
if not mains['INS_DIAM'].equals(mains['SEC_DIAM']):
    print("Warning: INS_DIAM and SEC_DIAM are not the same for each row.")

mask_no_additional_info_from_SEC_DIAM = mains.apply(lambda x: (x['INS_DIAM'] == x['SEC_DIAM']) or (x['SEC_DIAM'] is None) or (x['SEC_DIAM'] == 0), axis=1)
mask_no_additional_info_from_SEC_DIAM.value_counts(dropna=False)
# so need to save both INS_DIAM and SEC_DIAM for an edge

False    11070
True      2339
Name: count, dtype: int64

In [60]:
mains[['INS_DIAM', 'SEC_DIAM']]

,INS_DIAM,SEC_DIAM
0,250,0
1,750,None
2,750,None
3,750,None
4,750,None
...,...,...
13404,200,0
13405,750,0
13406,450,0
13407,250,0


 INS_DIAM (Installed Diameter)
Meaning: The actual internal diameter of the pipe that was installed.

Usage: This is the primary diameter relevant for hydraulic modeling, as it determines flow capacity and velocity.

Units: Likely in millimetres (mm), unless otherwise specified.

🔹 SEC_DIAM (Secondary Diameter)
Possible interpretations:

Segmented pipes: If the pipe has two different diameters along its length (e.g., due to rehab or staged installation), this could be the second diameter.

Non-circular geometry: Some culverts or elliptical pipes have a major and minor axis — this could represent the secondary (vertical or horizontal) dimension.

Redundancy tracking: In dual-pipe configurations (e.g., overflow or parallel relief mains), SEC_DIAM might denote a backup pipe size.

Planned vs. actual: Occasionally used to record a planned diameter alongside INS_DIAM as the installed one.

In [ ]:
manholes[manholes['GIS_ID'] == 'STMHFUTURE'] # planned manholes, not in the network, shouldn't be included in the network

,GIS_ID,FCODE,STATUS,DESC,LOCATION,PARENT,DESC_DTL1,DESC_DTL2,DESC_DTL3,INSTALL_DT,...,INS_DIAM,ASBUILT,MH_TYPE,RIM_ELEV,INV_ELEV,DFCDESGATE,DFCACTGATE,DFCTURNS,DFCDESFLOW,geometry
106,STMHFUTURE,STMHRGEX,NOT READY,"Manhole, Future Storm Manholes",DRAINAGEFUTURE,None,None,None,None,NaT,...,None,None,None,None,None,None,None,None,None,POINT (507859.229 5455961.698)
112,STMHFUTURE,STMHRGEX,NOT READY,"Manhole, Future Storm Manholes",DRAINAGEFUTURE,None,None,None,None,NaT,...,None,None,None,None,None,None,None,None,None,POINT (509703.405 5454082.161)
119,STMHFUTURE,STMHRGEX,NOT READY,"Manhole, Future Storm Manholes",DRAINAGEFUTURE,None,None,None,None,NaT,...,None,None,None,None,None,None,None,None,None,POINT (508270.949 5454999.858)
145,STMHFUTURE,STMHRGEX,NOT READY,"Manhole, Future Storm Manholes",DRAINAGEFUTURE,None,None,None,None,NaT,...,None,None,None,None,None,None,None,None,None,POINT (519194.44 5460070.526)
146,STMHFUTURE,STMHRGEX,NOT READY,"Manhole, Future Storm Manholes",DRAINAGEFUTURE,None,None,None,None,NaT,...,None,None,None,None,None,None,None,None,None,POINT (519164.861 5459920.782)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6282,STMHFUTURE,STMHRGEX,NOT READY,"Manhole, Future Storm Manholes",DRAINAGEFUTURE,None,None,None,None,NaT,...,None,None,None,None,None,None,None,None,None,POINT (508184.738 5454601.704)
6283,STMHFUTURE,STMHRGEX,NOT READY,"Manhole, Future Storm Manholes",DRAINAGEFUTURE,None,None,None,None,NaT,...,None,None,None,None,None,None,None,None,None,POINT (508348.118 5454722.884)
6409,STMHFUTURE,STMHRGEX,NOT READY,"Manhole, Future Storm Manholes",DRAINAGEFUTURE,None,None,None,None,NaT,...,None,None,None,None,None,None,None,None,None,POINT (519168.818 5459332.655)
6410,STMHFUTURE,STMHRGEX,NOT READY,"Manhole, Future Storm Manholes",DRAINAGEFUTURE,None,None,None,None,NaT,...,None,None,None,None,None,None,None,None,None,POINT (519154.354 5459294.656)


In [ ]:
mains[mains['DN_ID'] == 'STMHFUTURE'], mains[mains['UP_ID'] == 'STMHFUTURE'] # seems no mains with planned manholes as endpoints, so no need to remove them
# but is there planned mains? TODO: check if there are planned mains

(Empty GeoDataFrame
 Columns: [GIS_ID, FCODE, STATUS, DESC, LOCATION, PARENT, DESC_DTL1, DESC_DTL2, DESC_DTL3, INSTALL_DT, EXTRACT_DT, MAPSHEET, ASSET_ID, INS_DIAM, SEC_DIAM, GRADE, LENGTH, MAT_TYPE, ASBUILT, DN_TYPE, DN_ID, DN_ELEV, UP_TYPE, UP_ID, UP_ELEV, FLG_LINER, CROSS_RISK, geometry]
 Index: []
 
 [0 rows x 28 columns],
 Empty GeoDataFrame
 Columns: [GIS_ID, FCODE, STATUS, DESC, LOCATION, PARENT, DESC_DTL1, DESC_DTL2, DESC_DTL3, INSTALL_DT, EXTRACT_DT, MAPSHEET, ASSET_ID, INS_DIAM, SEC_DIAM, GRADE, LENGTH, MAT_TYPE, ASBUILT, DN_TYPE, DN_ID, DN_ELEV, UP_TYPE, UP_ID, UP_ELEV, FLG_LINER, CROSS_RISK, geometry]
 Index: []
 
 [0 rows x 28 columns])

# Construction of Graph

**not saving geometry for edges**

Justification and notes

In PINN/GNN Sewer Modeling, is geometry (LineString) necessary for edges?

✅ What is necessary:

Topology: Knowing that junction A connects to junction B via pipe E

Physical properties: e.g., length, diameter, slope

Boundary conditions or observations: like inflow at a node, or overflow at an outfall

🚫 What is not strictly necessary:
The full coordinate geometry (like a LineString) of the pipe, unless:

You’re doing high-resolution geospatial analysis (e.g., routing rainfall or pollutants along real-world paths)

You’re visualizing in GIS or integrating with map engines

You want to compute pipe length from geometry instead of trusting the attribute field

In [162]:
# get rid of future manholes
manholes = manholes[manholes['GIS_ID'] != 'STMHFUTURE']

In [167]:
# ─── Helper ──────────────────────────────────────────────────────────────────

def safe_val(val, name=None, node_or_edge=None):
    """
    Convert non-serializable or multi-value types into a single scalar/string:
      - pd.Series    → first element (if len>1, warn)
      - LineString   → WKT string
      - pd.Timestamp → ISO8601 string
      - None/NaN     → empty string
    """
    # 1) Unwrap Series
    if isinstance(val, pd.Series):
        if len(val) >= 1:
            v0 = val.iloc[0]
            if len(val) > 1:
                print(f"⚠️ Dropping {len(val)-1} extra values in Series for "
                      f"{node_or_edge} attribute '{name}'")
            val = v0
        else:
            return ""
    # 2) Convert geometry
    if isinstance(val, LineString):
        return val.wkt
    # 3) Convert timestamps
    if isinstance(val, pd.Timestamp):
        return val.isoformat()
    # 4) None or NaN → empty
    if val is None or (isinstance(val, float) and np.isnan(val)) or pd.isna(val):
        return ""
    # 5) Otherwise, primitive
    return val

def find_series_in_graph(G):
    issues = []
    for node, attrs in G.nodes(data=True):
        for k, v in attrs.items():
            if isinstance(v, pd.Series):
                issues.append(f"Node {node} attr '{k}'")
    for u, v, attrs in G.edges(data=True):
        for k, v0 in attrs.items():
            if isinstance(v0, pd.Series):
                issues.append(f"Edge {u}->{v} attr '{k}'")
    return issues


# Index by GIS_ID if available
if "GIS_ID" in manholes.columns:
    manholes = manholes.set_index("GIS_ID")

# ─── Build Graph ────────────────────────────────────────────────────────────

G = nx.DiGraph()

# Add nodes
for node_id, geom in manholes["geometry"].items():
    if node_id is None or geom is None or not hasattr(geom, "x"):
        continue
    row = manholes.loc[node_id]
    G.add_node(
        node_id,
        x          = float(geom.x),
        y          = float(geom.y),
        rim_elev   = safe_val(row.get("RIM_ELEV"),   "RIM_ELEV",   f"node {node_id}"),
        inv_elev   = safe_val(row.get("INV_ELEV"),   "INV_ELEV",   f"node {node_id}"),
        ins_diam   = safe_val(row.get("INS_DIAM"),   "INS_DIAM",   f"node {node_id}"),
        install_dt = safe_val(row.get("INSTALL_DT"), "INSTALL_DT", f"node {node_id}"),
        status     = safe_val(row.get("STATUS"),     "STATUS",     f"node {node_id}"),
        mh_type    = safe_val(row.get("MH_TYPE"),    "MH_TYPE",    f"node {node_id}"),
        dfc_desflow= safe_val(row.get("DFCDESFLOW"), "DFCDESFLOW", f"node {node_id}")
    )

# Add edges
for _, row in mains.iterrows():
    up = row.get("UP_ID"); dn = row.get("DN_ID")
    if pd.notna(up) and pd.notna(dn) and up in G and dn in G:
        G.add_edge(
            up, dn,
            length     = safe_val(row.get("LENGTH"),    "LENGTH",    f"edge {up}->{dn}"),
            ins_diam   = safe_val(row.get("INS_DIAM"),  "INS_DIAM",  f"edge {up}->{dn}"),
            sec_diam   = safe_val(row.get("SEC_DIAM"),  "SEC_DIAM",  f"edge {up}->{dn}"),
            grade      = safe_val(row.get("GRADE"),     "GRADE",     f"edge {up}->{dn}"),
            mat_type   = safe_val(row.get("MAT_TYPE"),  "MAT_TYPE",  f"edge {up}->{dn}"),
            install_dt = safe_val(row.get("INSTALL_DT"),"INSTALL_DT",f"edge {up}->{dn}"),
            status     = safe_val(row.get("STATUS"),    "STATUS",    f"edge {up}->{dn}"),
            fcode      = safe_val(row.get("FCODE"),     "FCODE",     f"edge {up}->{dn}")
        )

# ─── Validate ────────────────────────────────────────────────────────────────

n_nodes    = G.number_of_nodes()
n_mh       = manholes.index.nunique()
n_edges    = G.number_of_edges()
n_mains    = len(mains)

print(f"Graph nodes: {n_nodes}  (manholes: {n_mh})")
print(f"Graph edges: {n_edges}  (mains:     {n_mains})")

issues = find_series_in_graph(G)
if issues:
    print("\n⚠️ Still have pandas.Series in these attrs:")
    for i in issues:
        print("  -", i)
else:
    print("\n✅ No pandas.Series in graph; ready for GraphML export!")

⚠️ Dropping 1 extra values in Series for node STMH17065 attribute 'RIM_ELEV'
⚠️ Dropping 1 extra values in Series for node STMH17065 attribute 'INV_ELEV'
⚠️ Dropping 1 extra values in Series for node STMH17065 attribute 'INS_DIAM'
⚠️ Dropping 1 extra values in Series for node STMH17065 attribute 'INSTALL_DT'
⚠️ Dropping 1 extra values in Series for node STMH17065 attribute 'STATUS'
⚠️ Dropping 1 extra values in Series for node STMH17065 attribute 'MH_TYPE'
⚠️ Dropping 1 extra values in Series for node STMH17065 attribute 'DFCDESFLOW'
⚠️ Dropping 1 extra values in Series for node STMH17065 attribute 'RIM_ELEV'
⚠️ Dropping 1 extra values in Series for node STMH17065 attribute 'INV_ELEV'
⚠️ Dropping 1 extra values in Series for node STMH17065 attribute 'INS_DIAM'
⚠️ Dropping 1 extra values in Series for node STMH17065 attribute 'INSTALL_DT'
⚠️ Dropping 1 extra values in Series for node STMH17065 attribute 'STATUS'
⚠️ Dropping 1 extra values in Series for node STMH17065 attribute 'MH_TYPE'

There are duplicated manhole IDs

In [168]:
# check attributes of nodes and edges
def check_graph_attributes(G, limit=10):
    """Check and print attributes of nodes and edges in the graph."""
    print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
    print("\nNode attributes (first few nodes):")
    for i, (node, data) in enumerate(G.nodes(data=True)):
        if i >= limit:
            break
        print(f"Node {node}: {data}")
    print("\nEdge attributes (first few edges):")
    for i, (u, v, data) in enumerate(G.edges(data=True)):
        if i >= limit:
            break
        print(f"Edge from {u} to {v}: {data}")

# Check the graph attributes
check_graph_attributes(G)

Graph has 11146 nodes and 9876 edges.

Node attributes (first few nodes):
Node STMH11989: {'x': 512953.5007999996, 'y': 5460604.4387, 'rim_elev': '219.56', 'inv_elev': '217.81', 'ins_diam': '1050', 'install_dt': '1990-01-01T00:00:00', 'status': 'OPERATING', 'mh_type': 'STANDARD', 'dfc_desflow': ''}
Node STMH11985: {'x': 513151.07550000027, 'y': 5460509.6227, 'rim_elev': '212.93', 'inv_elev': '209.55', 'ins_diam': '1050', 'install_dt': '1990-01-01T00:00:00', 'status': 'OPERATING', 'mh_type': 'STANDARD', 'dfc_desflow': ''}
Node STMH11984: {'x': 513096.4360999996, 'y': 5460488.758300001, 'rim_elev': '210.65', 'inv_elev': '208.4', 'ins_diam': '1050', 'install_dt': '1990-01-01T00:00:00', 'status': 'OPERATING', 'mh_type': 'STANDARD', 'dfc_desflow': ''}
Node STMH05408: {'x': 507793.0203999998, 'y': 5456015.612400001, 'rim_elev': '999.99', 'inv_elev': '88.47', 'ins_diam': '1050', 'install_dt': '1968-06-01T00:00:00', 'status': 'DECOMMISSIONED', 'mh_type': 'STANDARD', 'dfc_desflow': ''}
Node STM

In [169]:
# save the networkx graph to a file
int_data_dir = "../int_data"
int_data_dir = os.path.join(int_data_dir, "storm_network.gpickle")
nx.write_graphml(G, int_data_dir)

In [170]:
# read back
G_loaded = nx.read_graphml(int_data_dir)
# Check if the loaded graph matches the original
assert G.number_of_nodes() == G_loaded.number_of_nodes(), "Node count mismatch after loading."
assert G.number_of_edges() == G_loaded.number_of_edges(), "Edge count mismatch after loading."
# Check if node attributes match
for node in G.nodes:
    assert G.nodes[node] == G_loaded.nodes[node], f"Node attributes mismatch for node {node} after loading."
print("Graph saved and loaded successfully with matching attributes.")


Graph saved and loaded successfully with matching attributes.


In [ ]:
# TODO: merge manholes (GIS_ID) and mains (UP_ID and DN_ID) to find missing nodes or edges
